<a href="https://colab.research.google.com/github/Dorin-Irimia/LLM_Google_Colab/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

# First model from GPT - 10 March


### 1️⃣ Instalează și importă librăriile

In [ ]:
!pip install transformers datasets torch pandas openpyxl

import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 2️⃣ Încarcă datele din Excel

In [ ]:
req_file = "Requirements.xlsx"
test_file = "Test_cases.xlsx"
req_df = pd.read_excel(req_file)
test_df = pd.read_excel(test_file)

# Verifică dacă fișierele sunt corect citite
print(req_df.head())
print(test_df.head())


   ID_REQ                            Requirement_Description        Status  \
0       1  The LIGHT shall be OFF when BUTTON detects ACT...  covered_less   
1       2  The MIRROR_STATUS shall be set to ACTIVE if th...  covered_less   
2       3  The FAN shall be DEPLOYED when CRASH_SENSOR de...   not_covered   
3       4  The WIPER_STATUS shall be set to ENABLED if th...   not_covered   
4       5  If SPEED_SENSOR is DEACTIVATED, then LIGHT_STA...   not_covered   

   ID_TEST Functionality  
0      1.0         LIGHT  
1      2.0        MIRROR  
2      NaN           FAN  
3      NaN         WIPER  
4      NaN         LIGHT  
   ID_REQ  ID_TEST                                   Test_Description  \
0       1        1  Verify functionality based on requirement: The...   
1       2        2  Verify functionality based on requirement: The...   
2       3        3  Verify functionality based on requirement: The...   
3       4        4  Verify functionality based on requirement: The...   
4   

### 3️⃣ Pregătește datele pentru antrenare

In [ ]:
# Merge requirements with test cases
data = req_df.merge(test_df, on="ID_REQ")

# Creare label map pentru clasificare
unique_labels = data["Test_Description"].unique()
label_map = {label: idx for idx, label in enumerate(unique_labels)}

def prepare_data(row):
    return {
        "text": row["Requirement_Description"],
        "label": label_map[row["Test_Description"]]
    }

dataset = [prepare_data(row) for _, row in data.iterrows()]
dataset = Dataset.from_pandas(pd.DataFrame(dataset))

# Verifică primele date pregătite
print(dataset[0])


{'text': 'The LIGHT shall be OFF when BUTTON detects ACTIVATED, else INACTIVE.', 'label': 0}


### 4️⃣ Încarcă modelul și tokenizer-ul

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_map))

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Verifică dacă tokenizarea funcționează
print(tokenized_datasets[0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

{'text': 'The LIGHT shall be OFF when BUTTON detects ACTIVATED, else INACTIVE.', 'label': 0, 'input_ids': [101, 1996, 2422, 4618, 2022, 2125, 2043, 6462, 11487, 2015, 8878, 1010, 2842, 16389, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### 5️⃣ Antrenează modelul

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"  # Dezactivează wandb
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-529f57c37a86>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,8.528900
1000,8.597200
1500,8.530500


TrainOutput(global_step=1875, training_loss=8.547720052083333, metrics={'train_runtime': 1495.3256, 'train_samples_per_second': 10.031, 'train_steps_per_second': 1.254, 'total_flos': 4122355138560000.0, 'train_loss': 8.547720052083333, 'epoch': 3.0})

### 6️⃣ Salvează modelul antrenat

In [ ]:
model.save_pretrained("./bert_finetuned")
tokenizer.save_pretrained("./bert_finetuned")

print("Model salvat cu succes!")


Model salvat cu succes!


### 7️⃣ Rulează inferența pe un fișier nou

In [ ]:
# Încarcă modelul antrenat
model = BertForSequenceClassification.from_pretrained("./bert_finetuned")
tokenizer = BertTokenizer.from_pretrained("./bert_finetuned")

def generate_test_cases(requirements):
    model.eval()
    test_cases = []

    for req in requirements:
        inputs = tokenizer(req, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_label = torch.argmax(logits, dim=1).item()

        test_description = [key for key, value in label_map.items() if value == predicted_label][0]
        test_cases.append(test_description)

    return test_cases

# Încarcă fișierul cu cerințe noi
new_req_df = pd.read_excel("Requirementss.xlsx")
requirements = new_req_df["Requirment_Description"].tolist()

# Generează testele
generated_test_cases = generate_test_cases(requirements)

# Salvează în Excel
results_df = pd.DataFrame({"Requirement": requirements, "Generated Test Case": generated_test_cases})
results_df.to_excel("Generated_Test_Cases.xlsx", index=False)

print("Fișierul 'Generated_Test_Cases.xlsx' a fost creat!")


Fișierul 'Generated_Test_Cases.xlsx' a fost creat!
